# Degree Planner Backend Algorithm

## TODO:

* In class Graph, use dict for nodes
* In class Graph, finish isCompleted()
* Figure out whats wrong with '__str__'
* Add all components to Node class (full name, description, etc.)
* In Graph, nodesToRemove(), return 2 lists: required, and possible to take
* Finish God


In [470]:
import copy 

In [438]:
class Node(object):
    name = ""
    credits = None
    prereq = []
    required = False
    completed = False
    description = ""
    fullName = ""
    priorityNumber = 0   # Number of edges to that node
    
    hardAdded = False
    
    offeredF = True
    offeredS = True
    offeredSu = False
    offeredEven = True
    offeredOdd = True
    
    def __init__(self, name, fullName, description, prereq, required, credits):
        self.name = name
        self.fullName = fullName
        self.description = description
        if prereq != None:
            self.prereq = prereq
        self.required = required
        self.credits = credits
        
    def __str__(self):
        if len(self.prereq) == 0:
            return str(self.name) + ' [] ' + str(self.required)
        retVal = self.name
        retVal += ' ['
        for item in self.prereq:
            retVal += (str(item.name) + ', ')
        retVal = retVal[:-2]
        retVal += '] '      
        retVal += str(self.required)
        return retVal
    
    def canComplete(self):
        if self.completed == True:
            return False
        for pr in self.prereq:
            if pr.completed == False:
                return False
        
        return True

In [412]:
class Graph(object):
    nodes = {}
    def __init__(self, nodesDict):
        self.nodes = nodesDict
        
    def __str__(self):
        return self.nodes
    
    def nodesToRemove(self): 
        retList = []
        for key, val in self.nodes.items():
            # print(key, val)
            if(val.canComplete() and val.hardAdded == False):
                retList.append(val)
        retList.sort(key=lambda x: x.priorityNumber, reverse=True)
        return retList
    
    def isCompleted(self): #Sets degree requirements
        for node in self.nodes:
            #print(str(node))
            if(not node.completed):
                return False
        return True
    
    def completeCourses(listOfCourses):
        for course in listOfCourses:
            for catalogCourse in nodes:
                if course.name == catalogCourse.name:
                    catalogCourse.completed = True
                    listOfCourses.remove(course)
        for course in listOfCourses:
            nodes.append(course)

In [654]:
class Semester:
    """
    Each student will have a set of semester objects. 
    """
    # available_classes = {} # computed through Node class
    classes_taken = {} # computed through Node class
    name = None # Fall, Spring, Summer
    year = None # '2018','2017'
    isEven = False
    credits_taken = 0 
    max_credits = 16
    
    def __init__(self, sem_name, sem_year, credits):
        self.name = sem_name
        self.year = sem_year
        self.max_credits = credits
        self.isEven = True if int(self.year[-1])% 2 == 0 else False
        self.reset()
    
    def reset(self):
        self.credits_taken = 0
        self.classes_taken = {}
        #print('T/M', self.credits_taken, self.max_credits)
        #print('reset done on', self.name, self.year)
    
    def canAdd(self, course):
        """
        Compute if the course can be taken during the given semester using the Graph object
        """
        
        #print(course.credits, self.credits_taken, self.max_credits)
        if (course.credits + self.credits_taken) > self.max_credits:
            print('Not enough credits')
            return False
        if course.offeredF and self.name[0] == 'F':
            if course.offeredEven and self.isEven: return True
            elif course.offeredOdd and not self.isEven: return True
            else : return False
        elif course.offeredS and self.name[0] == 'S':
            if course.offeredEven and self.isEven: return True
            elif course.offeredOdd and not self.isEven: return True
            else: return False
        elif course.offeredSu and self.name[0] == 'Su':
            if course.offeredEven and self.isEven: return True
            elif course.offeredOdd and not self.isEven: return True
            else: return False
        else: return False
    
    def add_course(self, course):  
        if self.canAdd(course):
            print('Added: ', course)
            self.classes_taken.update({course.name : course})
            self.credits_taken += course.credits
            course.completed = True
            return True
        return False

In [660]:
class God(object): #God algorithm!
    catalogYear = None
    allCourses = []
    terms = []
    graph = None
    allSemesters = []
    
    def __init__(self, catalogY):  #Possibly reinit lists!
        self.catalogYear = catalogY
        self.letThereBeLight()
        # print('Semseters: ', len(self.allSemesters))
        self.darwinEvolution()
        
        
        
    def darwinEvolution(self):
        # get courses from a specific year catalog
        self.allCourses = copy.deepcopy(testNodes) # TODO!!!
        self.graph = Graph(self.allCourses)
        
        for semester in self.allSemesters:
            print('\n*********** Semester: ', semester.name, semester.year, semester.max_credits)
            self.evolve(semester)
            

    
    def letThereBeLight(self):
        
        priorLearningSemester = Semester('PriorLearning', '0', 0)
        priorLearningSemester.max_credits = 0
        self.allSemesters.append(priorLearningSemester)
        
        catalogY = self.catalogYear 
        for year in range(catalogY*3, catalogY*3+24):
            #print(int(year/3))
            semName = None
            defaultCredits = 16
            if year % 3 == 0:
                semName = 'S'
            elif year % 3 == 1:
                semName = 'Su'
                defaultCredits = 0
            elif year % 3 == 2:
                semName = 'F'
            #print(int(year/3), semName, defaultCredits)
            #print(str(year/3))
            #print(defaultCredits)
            tempSemester = Semester(semName, str(int(year/3)), defaultCredits)
            #print(tempSemester.max_credits)
            self.allSemesters.append(tempSemester)
    
    # Choose high priority classes, then required, then possible to take
    def evolve(self, semester):
        #print('Evolve')
        semester.reset()
        tempList = self.graph.nodesToRemove()
        #print('List size: ', len(tempList))
        
        for course in tempList:
            print('Sending: ', course)
            addReturn = semester.add_course(course)
        
    def changeCredits(self, semseterNum, credits):
        self.allSemesters[semseterNum].max_credits = credits
        self.darwinEvolution()
        
    def semesterInfo(self):
        for semester in self.allSemesters:
            print('Semester: ', semester.name, semester.year, semester.credits_taken, '/', semester.max_credits)

In [661]:
# name, fullName, description, prereq, required, credits
MATH170 = Node('MATH170', 'CALCULUS I', 'Informal limits. Derivatives and antiderivatives.', None, True, 4)
CS121 = Node('CS121', 'COMPUTER SCIENCE I', 'Introduction to object-oriented problem solving and programming.', [MATH170], True, 4)
CS221 = Node('CS221', 'COMPUTER SCIENCE II', 'Object-oriented design including inheritance, polymorphism, and dynamic binding.', [CS121], True, 3)
CS321 = Node('CS321', 'DATA STRUCTURES', 'Sorting, searching, and order statistics.', [CS221], True, 3)
MATH189 = Node('MATH189', 'DISCRETE MATHEMATICS', 'Content drawn from propositional and predicate logic.', [MATH170], True, 4)




A.completed = True
testNodes = {'CS121':CS121, 'CS221':CS221, 'CS321':CS321, 'MATH170':MATH170, 'MATH189':MATH189}

catalogTest = Graph(testNodes)
catalogTest.nodes

{'CS121': <__main__.Node at 0x1f5312615f8>,
 'CS221': <__main__.Node at 0x1f5310f8ba8>,
 'CS321': <__main__.Node at 0x1f5310f8e80>,
 'MATH170': <__main__.Node at 0x1f5310f8710>,
 'MATH189': <__main__.Node at 0x1f5310f8908>}

In [662]:
god = God(2019)


*********** Semester:  PriorLearning 0 0
Sending:  MATH170 [] True
Not enough credits

*********** Semester:  S 2019 16
Sending:  MATH170 [] True
Added:  MATH170 [] True

*********** Semester:  Su 2019 0
Sending:  CS121 [MATH170] True
Not enough credits
Sending:  MATH189 [MATH170] True
Not enough credits

*********** Semester:  F 2019 16
Sending:  CS121 [MATH170] True
Added:  CS121 [MATH170] True
Sending:  MATH189 [MATH170] True
Added:  MATH189 [MATH170] True

*********** Semester:  S 2020 16
Sending:  CS221 [CS121] True
Added:  CS221 [CS121] True

*********** Semester:  Su 2020 0
Sending:  CS321 [CS221] True
Not enough credits

*********** Semester:  F 2020 16
Sending:  CS321 [CS221] True
Added:  CS321 [CS221] True

*********** Semester:  S 2021 16

*********** Semester:  Su 2021 0

*********** Semester:  F 2021 16

*********** Semester:  S 2022 16

*********** Semester:  Su 2022 0

*********** Semester:  F 2022 16

*********** Semester:  S 2023 16

*********** Semester:  Su 2023 0



In [663]:
god.changeCredits(3, 4)


*********** Semester:  PriorLearning 0 0
Sending:  MATH170 [] True
Not enough credits

*********** Semester:  S 2019 16
Sending:  MATH170 [] True
Added:  MATH170 [] True

*********** Semester:  Su 2019 0
Sending:  CS121 [MATH170] True
Not enough credits
Sending:  MATH189 [MATH170] True
Not enough credits

*********** Semester:  F 2019 4
Sending:  CS121 [MATH170] True
Added:  CS121 [MATH170] True
Sending:  MATH189 [MATH170] True
Not enough credits

*********** Semester:  S 2020 16
Sending:  CS221 [CS121] True
Added:  CS221 [CS121] True
Sending:  MATH189 [MATH170] True
Added:  MATH189 [MATH170] True

*********** Semester:  Su 2020 0
Sending:  CS321 [CS221] True
Not enough credits

*********** Semester:  F 2020 16
Sending:  CS321 [CS221] True
Added:  CS321 [CS221] True

*********** Semester:  S 2021 16

*********** Semester:  Su 2021 0

*********** Semester:  F 2021 16

*********** Semester:  S 2022 16

*********** Semester:  Su 2022 0

*********** Semester:  F 2022 16

*********** Semes

In [664]:
god.semesterInfo()

Semester:  PriorLearning 0 0 / 0
Semester:  S 2019 4 / 16
Semester:  Su 2019 0 / 0
Semester:  F 2019 4 / 4
Semester:  S 2020 7 / 16
Semester:  Su 2020 0 / 0
Semester:  F 2020 3 / 16
Semester:  S 2021 0 / 16
Semester:  Su 2021 0 / 0
Semester:  F 2021 0 / 16
Semester:  S 2022 0 / 16
Semester:  Su 2022 0 / 0
Semester:  F 2022 0 / 16
Semester:  S 2023 0 / 16
Semester:  Su 2023 0 / 0
Semester:  F 2023 0 / 16
Semester:  S 2024 0 / 16
Semester:  Su 2024 0 / 0
Semester:  F 2024 0 / 16
Semester:  S 2025 0 / 16
Semester:  Su 2025 0 / 0
Semester:  F 2025 0 / 16
Semester:  S 2026 0 / 16
Semester:  Su 2026 0 / 0
Semester:  F 2026 0 / 16
